In [2]:
!pip install geopandas -q

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', None)  # 無限行
pd.set_option('display.max_columns', None)  # 無限列

In [4]:
import pandas as pd
import geopandas as gpd
import os

# 定義函數區域
def read_houses_data(houses_path, count_columns, houses_df=None):
    if houses_df is None:
        houses_df = pd.read_csv(houses_path)
    else:
        houses_df = houses_df.copy()
    
    print("房屋資料經緯度前幾行:")
    print(houses_df[['經度', '緯度']].head())
    
    columns_to_process = []
    for count_column in count_columns:
        if count_column not in houses_df.columns:
            houses_df[count_column] = 0
            columns_to_process.append(count_column)
        else:
            if houses_df[count_column].notna().all():
                print(f"欄位 '{count_column}' 已計算完成，跳過計算。")
            else:
                houses_df[count_column] = 0
                columns_to_process.append(count_column)
    return houses_df, columns_to_process

def read_facilities_data(facility_info):
    facilities_list = []
    for facility_path, count_column in facility_info:
        facilities_df = pd.read_csv(facility_path, encoding='BIG5', skiprows=1)
        print(f"設施資料 '{facility_path}' 經緯度前幾行:")
        print(facilities_df[['WGS84']].head())
        
        facilities_df[['longitude', 'latitude']] = facilities_df['WGS84'].str.split(',', expand=True)
        facilities_df['longitude'] = facilities_df['longitude'].astype(float)
        facilities_df['latitude'] = facilities_df['latitude'].astype(float)
        facilities_df['facility_type'] = count_column
        
        facilities_list.append(facilities_df)
    combined_facilities_df = pd.concat(facilities_list, ignore_index=True)
    return combined_facilities_df

def create_geodataframe(df, x_col, y_col, crs='EPSG:4326'):
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[x_col], df[y_col]),
        crs=crs
    )
    return gdf

def project_geodataframe(gdf, target_crs='EPSG:3826'):
    return gdf.to_crs(target_crs)

def create_buffers(gdf, buffer_distances):
    for distance in buffer_distances:
        gdf[f'buffer_{distance}'] = gdf.geometry.buffer(distance)
    return gdf

def spatial_join_count(facilities_gdf, houses_gdf, buffer_distances):
    results = {}
    previous_distance = 0
    for distance in buffer_distances:
        buffer_col = f'buffer_{distance}'
        # 創建當前距離的緩衝區
        houses_gdf[f'buffer_{distance}'] = houses_gdf.geometry.buffer(distance)
        
        if previous_distance > 0:
            # 創建環狀區域（當前緩衝區減去上一個緩衝區）
            annulus = houses_gdf[f'buffer_{distance}'].difference(houses_gdf[f'buffer_{previous_distance}'])
            buffer_gdf = houses_gdf.copy()
            buffer_gdf['geometry'] = annulus
        else:
            # 第一個緩衝區直接使用
            buffer_gdf = houses_gdf.copy()
            buffer_gdf['geometry'] = houses_gdf[f'buffer_{distance}']
        
        # 移除空的幾何圖形
        buffer_gdf = buffer_gdf[buffer_gdf.geometry.notnull()]
        
        # 確保 buffer_gdf 是 GeoDataFrame
        buffer_gdf = gpd.GeoDataFrame(buffer_gdf, geometry='geometry', crs=houses_gdf.crs)
        
        # 執行空間聯結
        joined = gpd.sjoin(facilities_gdf, buffer_gdf, how='left', predicate='within')
        
        if joined.empty:
            print(f"沒有任何設施落在房屋的緩衝區（{previous_distance}-{distance}m）內")
            previous_distance = distance
            continue
        
        # 按照房屋索引和設施類型進行分組並計算數量
        count_column = f'facility_count_{previous_distance}_{distance}'
        facility_counts = joined.groupby(['index_right', 'facility_type']).size().reset_index(name=count_column)
        
        results[count_column] = facility_counts
    
        previous_distance = distance
    
    # 合併所有計數結果
    if not results:
        return pd.DataFrame()
    
    # 初始化結果 DataFrame
    all_counts = None
    for count_column, df in results.items():
        if all_counts is None:
            all_counts = df
        else:
            all_counts = pd.merge(all_counts, df, on=['index_right', 'facility_type'], how='outer')
    
    # 將 NaN 填充為 0
    all_counts.fillna(0, inplace=True)
    
    return all_counts

def update_houses_with_counts(houses_df, houses_gdf, facility_counts, count_columns, buffer_distances):
    # 為每個設施類型和距離範圍創建新的計數欄位
    for count_column in count_columns:
        for i in range(len(buffer_distances)):
            if i == 0:
                lower = 0
            else:
                lower = buffer_distances[i-1]
            upper = buffer_distances[i]
            new_col = f"{count_column}_{lower}_{upper}"
            houses_df[new_col] = 0
    
    if not facility_counts.empty:
        for _, row in facility_counts.iterrows():
            house_idx = row['index_right']
            facility_type = row['facility_type']
            for distance in buffer_distances:
                if distance == buffer_distances[0]:
                    lower = 0
                else:
                    lower = buffer_distances[buffer_distances.index(distance)-1]
                upper = distance
                count_col = f'facility_count_{lower}_{upper}'
                target_col = f"{facility_type}_{lower}_{upper}"
                # 初始化欄位（如果不存在）
                if target_col not in houses_df.columns:
                    houses_df[target_col] = 0
                # 累加計數
                houses_df.at[house_idx, target_col] += row[count_col]
    
    return houses_df

def save_results(houses_df, houses_path):
    filename, file_extension = os.path.splitext(houses_path)
    output_path = f'{filename}_with_counts{file_extension}'
    houses_df.to_csv(output_path, index=False)
    print(f"結果已導出至 {output_path}")
    return output_path

def compare_facilities(houses_path, facility_info, buffer_distances=[500, 1000, 1500], houses_df=None):
    # 根據設施資料生成具體距離範圍的計數欄位名稱
    count_columns = []
    for info in facility_info:
        facility_type = info[1]
        for i in range(len(buffer_distances)):
            if i == 0:
                lower = 0
            else:
                lower = buffer_distances[i-1]
            upper = buffer_distances[i]
            count_columns.append(f"{facility_type}_{lower}_{upper}")

    # 讀取房屋資料
    houses_df, columns_to_process = read_houses_data(houses_path, count_columns)

    # 即使所有計數欄位都已存在，也強制重新計算
    columns_to_process = count_columns

    if not columns_to_process:
        print("所有設施計數已完成，跳過計算。")
        return houses_df[['編號'] + count_columns + ['緯度', '經度']]
    
    # 創建房屋的 GeoDataFrame
    houses_gdf = create_geodataframe(houses_df, '經度', '緯度')
    
    # 讀取並合併設施資料
    facilities_df = read_facilities_data(facility_info)
    facilities_gdf = create_geodataframe(facilities_df, 'longitude', 'latitude')
    
    # 投影到目標座標系統
    houses_gdf = project_geodataframe(houses_gdf)
    facilities_gdf = project_geodataframe(facilities_gdf)
    
    # 創建緩衝區
    houses_gdf = create_buffers(houses_gdf, buffer_distances)
    
    # 執行空間聯結並計算設施數量
    facility_counts = spatial_join_count(facilities_gdf, houses_gdf, buffer_distances)
    
    # 更新房屋資料中的設施數量
    houses_df = update_houses_with_counts(houses_df, houses_gdf, facility_counts, [info[1] for info in facility_info], buffer_distances)
    
    # 移除不必要的緩衝區欄位
    houses_df = houses_df.drop(columns=[f'buffer_{d}' for d in buffer_distances], errors='ignore')
    
    # 儲存結果
    output_path = save_results(houses_df, houses_path)
    
    return houses_df

# 執行區域
# 設定房屋資料路徑
houses_path = "/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/實價登錄房價_237832row_cleaned.csv"

# 設定設施資料資訊（路徑, 計數欄位）
facility_info = [
    ('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/tainan50_houseGoodthings_sorted.csv', 'good_count'),
    ('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/tainan50_houseBadthings_sorted.csv', 'bad_count')
]

# 設定緩衝距離（單位：公尺）
buffer_distances = [500, 1000, 1500]

# 呼叫 compare_facilities 函數進行計算
houses_df = compare_facilities(houses_path, facility_info, buffer_distances=buffer_distances)

# 檢視結果的前幾行
print(houses_df.head())


房屋資料經緯度前幾行:
           經度         緯度
0  120.193697  22.994487
1  120.228436  23.008977
2  120.192413  23.188924
3  120.304210  23.045320
4  120.200844  22.985457
設施資料 '/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/tainan50_houseGoodthings_sorted.csv' 經緯度前幾行:
                  WGS84
0  120.252915,23.126531
1  120.218906,23.080076
2  120.225087,23.098491
3  120.233485,23.115057
4  120.246460,23.123788
設施資料 '/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/tainan50_houseBadthings_sorted.csv' 經緯度前幾行:
                  WGS84
0  120.253559,23.128710
1  120.261147,23.128563
2  120.253601,23.130280
3  120.239178,23.115250
4  120.246463,23.119887
結果已導出至 /Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/實價登錄房價_237832row_cleaned_with_counts.csv
  鄉鎮市區               土地位置建物門牌       交易年月日    建築完成年月  建物移轉總面積平方公尺 有無管理組織  \
0  中西區  臺南市中西區金華路三段２１８號十五樓之１７  2022-09-02  840704.0        43.51      有   
1   北區     臺南市北區南園街１２７巷１５１弄３號  2022-08-01  640322.0        69.20      無   
2  佳里區        臺南市佳里區麻興路二段５３１號

In [5]:
houses_df.columns

Index(['鄉鎮市區', '土地位置建物門牌', '交易年月日', '建築完成年月', '建物移轉總面積平方公尺', '有無管理組織', '總價元',
       '單價元平方公尺', '備註', '編號', '緯度', '經度', 'bad_count', 'good_count', '建築型態清整',
       '陽台有無', '是否含額外土地', '建築型態_住商大樓', '建築型態_公寓', '建築型態_其他', '建築型態_透天厝',
       '房地總價元', '格局比例_房', '格局比例_廳', '格局比例_衛', '格局比例_隔間', '是否包含車位', '建材_磚石',
       '建材_鋼筋', '建材_鋼骨', '建材_竹木', '住', '商', '工', '農', '移轉層次_騎樓', '移轉層次_屋頂',
       '移轉層次_一二樓', 'KDE_0.5km', 'KDE_1km', 'KDE_1.5km', '有無電梯', 'Region',
       '建物移轉總面積坪', '單價元每坪', '房地總價元(坪)', '交易年份', '建築完成年', '交易年_民國', '屋齡',
       '屋齡區間', 'good_count_0_500', 'good_count_500_1000',
       'good_count_1000_1500', 'bad_count_0_500', 'bad_count_500_1000',
       'bad_count_1000_1500'],
      dtype='object')

In [6]:
houses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237832 entries, 0 to 237831
Data columns (total 57 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   鄉鎮市區                  237832 non-null  object 
 1   土地位置建物門牌              237832 non-null  object 
 2   交易年月日                 237832 non-null  object 
 3   建築完成年月                212559 non-null  float64
 4   建物移轉總面積平方公尺           237832 non-null  float64
 5   有無管理組織                237832 non-null  object 
 6   總價元                   237832 non-null  int64  
 7   單價元平方公尺               237832 non-null  float64
 8   備註                    65379 non-null   object 
 9   編號                    237832 non-null  object 
 10  緯度                    237832 non-null  float64
 11  經度                    237832 non-null  float64
 12  bad_count             237832 non-null  float64
 13  good_count            237832 non-null  float64
 14  建築型態清整                237832 non-null  object 
 15  

In [7]:
houses_df.columns

Index(['鄉鎮市區', '土地位置建物門牌', '交易年月日', '建築完成年月', '建物移轉總面積平方公尺', '有無管理組織', '總價元',
       '單價元平方公尺', '備註', '編號', '緯度', '經度', 'bad_count', 'good_count', '建築型態清整',
       '陽台有無', '是否含額外土地', '建築型態_住商大樓', '建築型態_公寓', '建築型態_其他', '建築型態_透天厝',
       '房地總價元', '格局比例_房', '格局比例_廳', '格局比例_衛', '格局比例_隔間', '是否包含車位', '建材_磚石',
       '建材_鋼筋', '建材_鋼骨', '建材_竹木', '住', '商', '工', '農', '移轉層次_騎樓', '移轉層次_屋頂',
       '移轉層次_一二樓', 'KDE_0.5km', 'KDE_1km', 'KDE_1.5km', '有無電梯', 'Region',
       '建物移轉總面積坪', '單價元每坪', '房地總價元(坪)', '交易年份', '建築完成年', '交易年_民國', '屋齡',
       '屋齡區間', 'good_count_0_500', 'good_count_500_1000',
       'good_count_1000_1500', 'bad_count_0_500', 'bad_count_500_1000',
       'bad_count_1000_1500'],
      dtype='object')

In [9]:

df = pd.read_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/實價登錄房價_237832row_cleaned.csv') 

In [10]:
# 提取需要的列
df_facilities = houses_df[['編號','good_count_0_500', 'good_count_500_1000',
       'good_count_1000_1500', 'bad_count_0_500', 'bad_count_500_1000',
       'bad_count_1000_1500']]

# 合并 'Region' 列到 df
df = df.merge(df_facilities, on='編號', how='left')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237832 entries, 0 to 237831
Data columns (total 57 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   鄉鎮市區                  237832 non-null  object 
 1   土地位置建物門牌              237832 non-null  object 
 2   交易年月日                 237832 non-null  object 
 3   建築完成年月                212559 non-null  float64
 4   建物移轉總面積平方公尺           237832 non-null  float64
 5   有無管理組織                237832 non-null  object 
 6   總價元                   237832 non-null  int64  
 7   單價元平方公尺               237832 non-null  float64
 8   備註                    65379 non-null   object 
 9   編號                    237832 non-null  object 
 10  緯度                    237832 non-null  float64
 11  經度                    237832 non-null  float64
 12  bad_count             237832 non-null  float64
 13  good_count            237832 non-null  float64
 14  建築型態清整                237832 non-null  object 
 15  

In [11]:
df.to_csv('/Volumes/Debbie 1T/BDSE35-1_Final Project_2/data/實價登錄房價_237832row_cleaned_facilities.csv',index=False)